# comparing : 

* labmate Ai 
* Bayesien Optimization approch

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys 
import time
import random
import time
from numpy.linalg import inv
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.externals.joblib import dump
from sklearn import model_selection 
from sklearn.model_selection import GridSearchCV, cross_val_score,StratifiedKFold, learning_curve
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.model_selection import validation_curve,ShuffleSplit
from sklearn.preprocessing import scale
from sklearn.ensemble import RandomForestRegressor,  RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
from scipy.stats import norm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, Matern
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.calibration import calibration_curve
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from utils import *


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


/NOBACKUP/natika/mypython/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
df = pd.read_excel('5760_simple_discriptors-SMILES.xlsx')
data = df.drop([ 'Reaction_No', 'SMILES', 'Catalyst_1_Short_Hand','SMILES_R1','SMILES_R2','SMILES','SMILES_LI','SMILES_BASE','SMILES_SOLV'], axis=1 )
data_used = data.dropna(axis=0 , how='any')

# dropping missed values
data_used = data_used.reset_index().drop('index', axis=1).copy()

# Normalization of continuous variables 
data_used['Product_Yield_PCT_Area_UV'] = data_used['Product_Yield_PCT_Area_UV']/100

xls = pd.ExcelFile('Descriptors for Computational Modelling.xlsx')
df_Bases = pd.read_excel(xls, 'Base_Short_Hand')
df_Solvents = pd.read_excel(xls, 'Solvent_1_Short_Hand')
df_Ligands = pd.read_excel(xls, 'Ligand_Short_Hand')

# one hot encoding
data_ohe = data_cleaning(data_used)
Y_ohe = data_ohe["Product_Yield_PCT_Area_UV"]
data_ohe = data_ohe.drop('Product_Yield_PCT_Area_UV', axis=1)
X_ohe = pd.get_dummies(data_ohe)
data_ohe = X_ohe.copy()
data_ohe["Product_Yield_PCT_Area_UV"] = Y_ohe.copy()

# discreptors
df_descr = data_discreptors(data_used,xls,df_Ligands,df_Bases,df_Solvents)
Y_just_descri = df_descr["Product_Yield_PCT_Area_UV"]
df = df_descr.drop(['Product_Yield_PCT_Area_UV',"Ligand_Short_Hand","Base_Short_Hand","Solvent_1_Short_Hand"], axis=1)
X_just_descri = pd.get_dummies(df)
df_descr = X_just_descri.copy()
df_descr["Product_Yield_PCT_Area_UV"] = Y_just_descri.copy()

In [3]:
df_descr.shape

(3304, 26)

# Aquisition functions

In [4]:
def  cross_validations_plots(model,X,Y,n_splits = 4):
     
    #CV = [KFold,ShuffleSplit]
    CV = [ShuffleSplit]
    cv_results = []
    n_splits = 4
    for cv in CV :
        this_cv = cv(n_splits=n_splits)
        result = -cross_val_score(model, X, Y, cv=this_cv, scoring='neg_root_mean_squared_error') 
        cv_results.append(result)        
        
    cv_means = []
    cv_std = []
    for cv_result in cv_results:
        cv_means.append(cv_result.mean())
        cv_std.append(cv_result.std())

    cv_res = pd.DataFrame({"CrossValMeans":cv_means,"CrossValerrors": cv_std,"Algorithm":["ShuffleSplit"]})
    return cv_res

def expected_improvement_RF(model,X_unseen,X_train,y_train,xi=0.1):
    model.fit(X_train,y_train)
    dim = X_train.shape[1]
    
    all_predictions = []
    for e in model.estimators_:
        #print(X_unseen.shape)
        #print(X_train.shape)
        all_predictions += [e.predict(X_unseen)]
    
    variance = np.var(all_predictions, axis=0)
    sigma = np.sqrt(variance)
    mu = model.predict(X_unseen) 
    mu_sample = model.predict(X_train) 
    
    
    sigma = sigma.reshape(-1, X_unseen.shape[0])
    
    mu_sample_opt = np.max(mu_sample) 

    with np.errstate(divide='warn'): 
        imp = mu - mu_sample_opt - xi 
        Z = imp / sigma 
        ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z) 
        ei[sigma == 0.0] = 0.0 
    
    
    ei_df = pd.DataFrame(data=ei.reshape(-1,ei.shape[0]), columns=['expected_improvement_RF'],index = X_unseen.index)  
    predictions = model.predict(X_unseen)
    predictions_df = pd.DataFrame(data=predictions, columns=['Prediction'],index = X_unseen.index)
    assert len(ei_df) == len(predictions) # control line
    initial_data = pd.DataFrame(data=X_unseen, columns = list(X_unseen.columns.values),index = X_unseen.index)
    df = pd.concat([initial_data, predictions_df, ei_df], axis=1)
    return df 

def probability_of_improvement(model,X_unseen,X_train,y_train):
    model.fit(X_train,y_train)
    dim = X_train.shape[1]
    
    all_predictions = []
    for e in model.estimators_:
        all_predictions += [e.predict(X_unseen)]
    
    variance = np.var(all_predictions, axis=0)
    sigma = np.sqrt(variance)
    
    mu = model.predict(X_unseen) 
    mu_sample = model.predict(X_train) 
    
    
    sigma = sigma.reshape(-1, X_unseen.shape[0])
    
    mu_sample_opt = np.max(mu_sample) 

    with np.errstate(divide='warn'):          
        pi = norm.cdf((mu - mu_sample_opt)/(sigma+1E-9)) 
    
    
    pi_df = pd.DataFrame(data=pi.reshape(-1,pi.shape[0]), columns=['probability_of_improvement'])  
    predictions = model.predict(X_unseen)
    predictions_df = pd.DataFrame(data=predictions, columns=['Prediction'])
    assert len(pi_df) == len(predictions) # control line
    #initial_data = pd.DataFrame(data=X_unseen, columns = list(X_unseen.columns.values))
    #df = pd.concat([initial_data, predictions_df, pi_df], axis=1)
    df = pd.concat([predictions_df, pi_df], axis=1)
    return df

# test 
#model = RandomForestRegressor(200)
#X_train, X_test,y_train, y_test = train_test_split(X_just_descri, Y_just_descri, train_size=0.06, random_state = 2)
#pi_values = probability_of_improvement(model,X_test,X_train,y_train).iloc[:100,-1]
#plt.figure(figsize=(20,6))
#plt.plot(ei_values,'or', label='Expected improvement')
#plt.xlabel('Iteration')
#plt.ylabel(' yield')
#plt.legend()


def upper_confidence_bound(model,X_unseen,X_train,y_train,beta = 0.1):
    model.fit(X_train,y_train)
    dim = X_train.shape[1]
    
    all_predictions = []
    for e in model.estimators_:
        all_predictions += [e.predict(X_unseen)]
    
    variance = np.var(all_predictions, axis=0)
    sigma = np.sqrt(variance)
    
    mu = model.predict(X_unseen) 
    mu_sample = model.predict(X_train) 
    
    
    sigma = sigma.reshape(-1, X_unseen.shape[0])
    
    mu_sample_opt = np.max(mu_sample) 

    
    ucb = mu - beta*sigma # where beta > 0 is a tradeoff paramete
    
    
    ucb_df = pd.DataFrame(data=ucb.reshape(-1,ucb.shape[0]), columns=['upper_confidence_bound'])  
    predictions = model.predict(X_unseen)
    predictions_df = pd.DataFrame(data=predictions, columns=['Prediction'])
    assert len(ucb_df) == len(predictions) # control line
    #initial_data = pd.DataFrame(data=X_unseen, columns = list(X_unseen.columns.values))
    #df = pd.concat([initial_data, predictions_df, pi_df], axis=1)
    df = pd.concat([predictions_df, ucb_df], axis=1)
    return df

# Defining chemical space

In [5]:
#### Defining chemical space for simulating navigation of reaction space
class ChemicalSpace():
    ''' Abstract class of chemical space for simulating navigation of 
        reaction space using machine learning'''
    
    def __init__(self, df, seed=None):
        # seed for reproduciibity
        random.seed(seed)
        self.df = df
        self.index = list(df.index)
        self.explored_space_index = []
        self.space_len = len(self.index)
        # randomize the indexes
        random.shuffle(self.index)
        
        
    def random_guess(self,number_initial_data):
        '''Returns the dataframe with random rxns from chemical space'''
        num_of_rxns = number_initial_data    
        random_rxns_idxs = []        
        for idx  in self.index:
            if idx not in self.explored_space_index:
                self.explored_space_index.append(idx)
                random_rxns_idxs.append(idx)
                if len(random_rxns_idxs) == num_of_rxns:
                    break
        random_df = self.df.loc[random_rxns_idxs]
        return random_df
    
    
    def get_unused(self):
        ''' Return a dataframe of all reactions which has not been explored so far'''
        
        unused_rxn_idxs = []
        
        for idx in self.index:
            if idx not in self.explored_space_index:
                unused_rxn_idxs.append(idx)
                
        unused_rxns = self.df.loc[unused_rxn_idxs]
        return unused_rxns
        
    def get_explored(self):
        ''' Return a dataframe with indexes of reactions which have been chosen'''
        
        return self.df.loc[self.explored_space_index]
    
    def update_explored_rxns(self, rxn_idxs):
        ''' Add rxn_idxs to the list of explored reactions'''
        self.explored_space_index += rxn_idxs
        
    def is_empty(self):
        ''' Return True if the whole chemical space has been explored'''
        if len(self.explored_space_index) == self.space_len:
            return True
        else:
            return False
        
    def percent_explored(self):
        ''' Return the percent of exploration of chemical space'''
        return len(self.explored_space_index)/self.space_len


# Exploration of chemical space

In [6]:
#### Exploration of chemical space
class Simulation():
    def __init__(self, model, data,method,screen_size,number_initial_data,test_print,max_iter):
        self.data = data
        self.chemspace = ChemicalSpace(data, )
        self.method = method
        self.screen_size = screen_size
        self.max_real_yield = []
        self.number_initial_data = number_initial_data
        self.test_print = test_print
        self.max_iter = max_iter 
        self.RMSE_mean = []
        self.RMSE_sd = []
        self.model = model
        
 
    
    def explore_space(self): 
        
        # define the model                 
        model = self.model 
        
        # randomly guess select k reactions from chemical space
        random_guess = self.chemspace.random_guess(self.number_initial_data)        

        # Evaluate average yield of selected reactions
        self.max_real_yield = [np.mean(random_guess['Product_Yield_PCT_Area_UV'])]    
        
        # Evaluate the RMSE of the random selected reactions
        X_train , y_train = shuffle(random_guess.drop('Product_Yield_PCT_Area_UV', axis=1),
                                    random_guess['Product_Yield_PCT_Area_UV'])
        cv_res = cross_validations_plots(self.model, X_train , y_train,n_splits = 4)
        self.RMSE_mean.append(cv_res["CrossValMeans"].values) 
        self.RMSE_sd.append(cv_res["CrossValerrors"].values) 

        
        
        
        
        iteration = 1
        best_idxs = []
        
        
        while not self.chemspace.is_empty() and iteration < self.max_iter:
        #while not self.chemspace.is_empty():
            
            # split data for reactions explored so far           
            X_train , y_train = shuffle(self.chemspace.get_explored().drop('Product_Yield_PCT_Area_UV', axis=1),
                                        self.chemspace.get_explored()['Product_Yield_PCT_Area_UV'])
            
            
            #print(X_train.shape)
            # train the random forest on this data
            model.fit(X_train,y_train)
            

            
            
            # Get a dataframe with all reaction which hasn't been performed
            unseen = self.chemspace.get_unused()
                        
            X_unseen = unseen.drop('Product_Yield_PCT_Area_UV', axis=1).copy()
            
            
            ##################################
            # choose the exploration strategy#   
            ##################################
            
            if (self.method == 'Labmate_Ai'):                
                yp = model.predict(X_unseen)
                unseen[self.method] = yp
                sorted_by = unseen.sort_values(by=[self.method],ascending=False)
                                                
            if (self.method == 'expected_improvement_RF'):
                #print(X_unseen.shape)
                unseen[self.method] = expected_improvement_RF(model,X_unseen,X_train,y_train,xi=0.01)[self.method].values
                sorted_by = unseen.sort_values(by=[self.method],ascending=False)

            
            if (self.method == 'probability_of_improvement'):
                unseen[self.method] = probability_of_improvement(model,X_unseen,X_train,y_train)[self.method].values
                sorted_by = unseen.sort_values(by=[self.method],ascending=False)
                
            if (self.method == "random_search"):
                unseen.sample(n=len(unseen))
                unseen[self.method] = unseen['Product_Yield_PCT_Area_UV'].values
                sorted_by = unseen

            if (self.method == "upper_confidence_bound"):
                unseen[self.method] = upper_confidence_bound(model,X_unseen,X_train,y_train,beta=0.1)[self.method].values
                sorted_by = unseen.sort_values(by=[self.method],ascending=False)
                

                    
            # Get a dataframe with best candidates from method chosen.
            best_results = sorted_by.head(self.screen_size)
                        
            # Get idxs of best candidates 
            best_results_idxs = list(best_results.index) 
            best_idxs = best_idxs + best_results_idxs    
            
            # Get idxs of best candidates 
            best_results_idxs = list(best_results.index) 
                                                            
            # Evaluate real yiled of selected batch of reactions 
            f_star = np.mean(best_results['Product_Yield_PCT_Area_UV'])
            self.max_real_yield.append(f_star)           
            
            
            # Evaluate the RMSE of the selected batches so far
            cv_res = cross_validations_plots(self.model,X_train , y_train,n_splits = 4)
            self.RMSE_mean.append(cv_res["CrossValMeans"].values) 
            self.RMSE_sd.append(cv_res["CrossValerrors"].values) 
            

            
            
            

            # Add the current batch of reactions to explored reactions so in the next iteration RF can be trained on updated data    
            self.chemspace.update_explored_rxns(best_results_idxs)
                        
                
          
            if self.test_print == True:
                if iteration % 10 == 0:
                    print(f" the yield at iteration {iteration} is {f_star}")
                    print(f" X_train.shape : {X_train.shape} ")
                    print(f" X_test.shape  : {X_unseen.shape} ")
            
            iteration += 1
        
        
        
                                    
        # Create a data frame 
        stat_dataframe = pd.DataFrame()
        stat_dataframe['avg_real_yield'] = self.max_real_yield  
        stat_dataframe['RMSE_mean'] = self.RMSE_mean
        stat_dataframe['RMSE_sd'] = self.RMSE_sd

        return stat_dataframe


# Comparing : 
- LAbmateAi  
- upper bound 


# Simulations & Averaving   descriptors Vs OHE

In [7]:
def simulations(n_simu = 10):
    test_print = False
    max_iter = 20
    screen_size = 5
    number_initial_data = 5
    model = RandomForestRegressor(300,n_jobs = -1)
    
    Av_labmateai_y = []
    Av_ucb_y = []


    Av_labmateai_rmse = []
    Av_ucb_rmse = []
    
   
    Av_labmateai_y_ohe = []
    Av_ucb_y_ohe = []


    Av_labmateai_rmse_ohe = []
    Av_ucb_rmse_ohe = []
    
    
    
    k = 1
    t1 = time.time()
    for _ in range(n_simu):
        t = time.time()
        print(f" iteration {k}")
        
        ########################
        # descriptors
        ########################
        
        # labmateai
        method = "Labmate_Ai"
        simulation_labmateai = Simulation(model,df_descr,method,screen_size,number_initial_data,test_print,max_iter)
        stat_dataframe_labmateai = simulation_labmateai.explore_space()
        avg_real_yield_labmateai = stat_dataframe_labmateai['avg_real_yield']
        RMSE_mean_labmateai = stat_dataframe_labmateai['RMSE_mean']

        # upper_confidence_bound
        method = "upper_confidence_bound"
        simulation_UCB = Simulation(model,df_descr,method,screen_size,number_initial_data,test_print,max_iter)
        stat_dataframe_UCB = simulation_UCB.explore_space()
        avg_real_yield_UCB = stat_dataframe_UCB['avg_real_yield']
        RMSE_mean_UCB = stat_dataframe_UCB['RMSE_mean']

        Av_labmateai_y.append(avg_real_yield_labmateai)
        Av_ucb_y.append(avg_real_yield_UCB)        
        
        Av_labmateai_rmse.append(RMSE_mean_labmateai)
        Av_ucb_rmse.append(RMSE_mean_UCB)        
        
        
        
        ########################
        # OHE
        ########################
        
        
        # labmateai
        method = "Labmate_Ai"
        simulation_labmateai = Simulation(model,data_ohe,method,screen_size,number_initial_data,test_print,max_iter)
        stat_dataframe_labmateai = simulation_labmateai.explore_space()
        avg_real_yield_labmateai_ohe = stat_dataframe_labmateai['avg_real_yield']
        RMSE_mean_labmateai_ohe = stat_dataframe_labmateai['RMSE_mean']

        # upper_confidence_bound
        method = "upper_confidence_bound"
        simulation_UCB = Simulation(model,data_ohe,method,screen_size,number_initial_data,test_print,max_iter)
        stat_dataframe_UCB = simulation_UCB.explore_space()
        avg_real_yield_UCB_ohe = stat_dataframe_UCB['avg_real_yield']
        RMSE_mean_UCB_ohe = stat_dataframe_UCB['RMSE_mean']        
        
        
        Av_labmateai_y_ohe.append(avg_real_yield_labmateai)
        Av_ucb_y_ohe.append(avg_real_yield_UCB)
                
        Av_labmateai_rmse_ohe.append(RMSE_mean_labmateai)
        Av_ucb_rmse_ohe.append(RMSE_mean_UCB)

        print( "time : " + str(( time.time()-t)/60) + " min")
            
        k += 1
        
        
    t2 = time.time()
    print( "time : " + str((t2-t1)/60) + " min")
        
        
        
    # descriptors    
    av_labmateai_y = np.mean(np.array(Av_labmateai_y),axis =0)
    av_ucb_y = np.mean(np.array(Av_ucb_y),axis =0)
    av_labmateai_rmse = np.mean(np.array(Av_labmateai_rmse),axis =0)
    av_ucb_rmse = np.mean(np.array(Av_ucb_rmse),axis =0)
    
    
    # ohe 
    av_labmateai_y_ohe = np.mean(np.array(Av_labmateai_y_ohe),axis =0)
    av_ucb_y_ohe = np.mean(np.array(Av_ucb_y_ohe),axis =0)    
    av_labmateai_rmse_ohe = np.mean(np.array(Av_labmateai_rmse_ohe),axis =0)
    av_ucb_rmse_ohe = np.mean(np.array(Av_ucb_rmse_ohe),axis =0)

    
    
    ###############################################################    
    # plot  : All strategies : yiels & RMSE averaging
    ###############################################################    
    
    xfit = np.arange(number_initial_data,number_initial_data + max_iter*screen_size , screen_size)

    plt.figure(figsize=(35,6))
    plt.plot(xfit,av_labmateai_rmse, '-', label='RMSE labmateai descriptors')
    plt.plot(xfit,av_ucb_rmse, '-', label='RMSE UCB descriptors')
    plt.plot(xfit,av_labmateai_rmse_ohe, '-', label='RMSE labmateai ohe')
    plt.plot(xfit,av_ucb_rmse_ohe, '-', label='RMSE UCB ohe')
    
    
    plt.xlabel('Reactions')
    plt.ylabel('The average of the updated RMSE after every selection ')
    plt.legend()
    plt.pause(0.1)

    
    
    
    
    
    plt.figure(figsize=(35,6))
    plt.plot(xfit,av_labmateai_y, '-', label='Average yield per batch : labmateai descriptors')
    plt.plot(xfit,av_ucb_y, '-', label='Average yield per batch : UCB descriptors')
    plt.plot(xfit,av_labmateai_y_ohe, '-', label='Average yield per batch : labmateai ohe')
    plt.plot(xfit,av_ucb_y_ohe, '-', label='Average yield per batch : UCB ohe')
    plt.xlabel('Reactions')
    plt.ylabel('the Average of the Averages of true yield in every batch ')
    plt.legend()

In [ ]:
simulations(n_simu = 2)

 iteration 1
